In [155]:
from setup._order import simulate_from_order_func
from sklearn.model_selection import train_test_split
from utils._utils import get_csv_data
import pandas as pd

In [156]:
opens = get_csv_data("data/crypto_open_data.csv")
closes = get_csv_data("data/crypto_close_data.csv")

train_opens, opens = train_test_split(opens, test_size=0.20, train_size=0.80, shuffle=False)
train_closes, closes = train_test_split(closes, test_size=0.20, train_size=0.80, shuffle=False)

print(closes.shape[0])

254304


In [ ]:
# Constants
# 120 2.4 -2.0 1.0 1.000000e-08 0.8

PERIOD = 26500
UPPER = 3.3
LOWER = -4.0
EXIT = 0.7
DELTA = 1e-3
VT = 0.8
BURNIN = 500

In [163]:
pf = simulate_from_order_func(
    closes, opens, 
    period=PERIOD,
    upper=UPPER,
    lower=LOWER,
    exits=EXIT,
    delta=DELTA,
    vt=VT,
    burnin=BURNIN,
    freq="h",
    hedge="beta",
)

In [164]:
pf.total_return()

-0.009836671971281522

In [165]:
res = pf.trades.records_readable.groupby("Entry Timestamp").sum()
res.PnL[res.PnL > 0].shape[0] / res.shape[0]

0.3333333333333333